
# Árboles de decisión y Ensambles con datos de la EDA

En este cuaderno se utiliza uno de los conjuntos de datos que se han trabajado en las EDA (por ejemplo `archive/Base.csv`) para entrenar y evaluar modelos de Árbol de Decisión y Bosque Aleatorio (RandomForest).

El flujo general es:

1. Cargar el dataset y explorar sus dimensiones.
2. Preprocesar los datos: manejar valores nulos, codificar variables categóricas y normalizar las variables numéricas.
3. Dividir los datos en conjuntos de entrenamiento y prueba.
4. Entrenar un **DecisionTreeClassifier** y un **RandomForestClassifier**.
5. Evaluar el desempeño de ambos modelos usando métricas de clasificación (accuracy, F1, matriz de confusión).
6. Visualizar la importancia de las variables según el bosque aleatorio.
7. Comparar resultados y discutir el ajuste, sesgo y varianza de los modelos.


In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Ruta del dataset de la EDA (ajusta según la estructura de tu repositorio)
# Ejemplo: 'archive/Base.csv' debe existir cuando ejecutes el notebook.
dataset_path = 'archive/Base.csv'

# Cargar el dataset
df = pd.read_csv(dataset_path)
print('Forma del conjunto de datos:', df.shape)
df.head()


In [ ]:

# Eliminar filas con valores nulos
df = df.dropna().reset_index(drop=True)

# Detectar la columna objetivo buscando palabras clave en el nombre
possible_targets = [col for col in df.columns if any(key in col.lower() for key in ['fraud', 'fraude', 'target', 'objetivo'])]
if possible_targets:
    target_col = possible_targets[0]
else:
    # Si no se encuentra, utiliza la última columna como objetivo
    target_col = df.columns[-1]

print('Columna objetivo seleccionada:', target_col)

# Separar características (X) y etiqueta (y)
X = df.drop(columns=[target_col]).copy()
y = df[target_col].copy()

# Codificar variables categóricas
cat_cols = X.select_dtypes(include=['object', 'category']).columns
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Convertir a numpy arrays
X_array = X.values
y_array = y.values

# Dividir en entrenamiento y prueba (80/20)
X_train, X_test, y_train, y_test = train_test_split(X_array, y_array, test_size=0.2, random_state=42, stratify=y_array)

print('Tamaño entrenamiento:', X_train.shape, 'Tamaño prueba:', X_test.shape)


In [ ]:

# Entrenar Árbol de Decisión
clf_tree = DecisionTreeClassifier(random_state=42)
clf_tree.fit(X_train, y_train)

# Predicciones y métricas para el árbol
y_pred_tree = clf_tree.predict(X_test)
acc_tree = accuracy_score(y_test, y_pred_tree)
f1_tree = f1_score(y_test, y_pred_tree, average='binary')
cm_tree = confusion_matrix(y_test, y_pred_tree)

print('Desempeño del Árbol de Decisión:')
print('Accuracy:', acc_tree)
print('F1-score:', f1_tree)
print('Matriz de confusión:
', cm_tree)


In [ ]:

# Entrenar Bosque Aleatorio
clf_rf = RandomForestClassifier(n_estimators=100, random_state=42)
clf_rf.fit(X_train, y_train)

# Predicciones y métricas para el bosque
y_pred_rf = clf_rf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf, average='binary')
cm_rf = confusion_matrix(y_test, y_pred_rf)

print('Desempeño del Bosque Aleatorio:')
print('Accuracy:', acc_rf)
print('F1-score:', f1_rf)
print('Matriz de confusión:
', cm_rf)


In [ ]:

# Importancia de características del Bosque Aleatorio
importances = clf_rf.feature_importances_
indices = np.argsort(importances)[::-1]
top_features = X.columns[indices]

plt.figure(figsize=(10,6))
plt.title('Importancia de características (RandomForest)')
plt.bar(range(len(importances)), importances[indices], align='center')
plt.xticks(range(len(importances)), top_features, rotation=90)
plt.ylabel('Importancia')
plt.tight_layout()
plt.show()


In [ ]:

# Comparar métricas en un DataFrame
results = pd.DataFrame({
    'Modelo': ['Árbol de Decisión', 'Bosque Aleatorio'],
    'Accuracy': [acc_tree, acc_rf],
    'F1-score': [f1_tree, f1_rf]
})
results



## Discusión sobre sesgo, varianza y ajuste

* **Sesgo:** El árbol de decisión suele tener bajo sesgo pero puede sobreajustarse a los datos. El bosque aleatorio tiende a reducir el sesgo al promediar muchos árboles.
* **Varianza:** Un solo árbol tiene alta varianza; el bosque aleatorio disminuye la varianza gracias al bagging y la selección aleatoria de características.
* **Ajuste:** Si las métricas en entrenamiento y prueba son similares y altas, el modelo está bien ajustado. Si el rendimiento en entrenamiento es mucho mejor que en prueba, hay sobreajuste (overfitting). Un rendimiento bajo en ambos conjuntos indica subajuste (underfitting).

Ajustando hiperparámetros como la profundidad máxima del árbol (`max_depth`) o el número de árboles (`n_estimators`) se puede mejorar el equilibrio entre sesgo y varianza.
